In [ ]:
from Solve import psi
import numpy as np
n = 12000
wavefunction, x, t = psi([-200, 4096, -300, n])

import matplotlib.pyplot as plt
for i in range(0, n, 10**5):
    plt.plot(x, abs(wavefunction[i]))
plt.xlim(-40, 40)
#plt.ylim(0, 5)
plt.show()

MemoryError: Unable to allocate 61.0 GiB for an array with shape (1000001, 4096) and data type complex128

In [ ]:
from Res_visualisation import *
from Parameters import pars_YanPengPhysRevA_78_033821
from Hydrogen import Hydrogen
from Field import Field

params = pars_YanPengPhysRevA_78_033821()[0]
H = Hydrogen()
F = Field.Pulse(params)
field = F(t, 'Real')
pot = H.potential(x)
plot_HH_spectrum(x, t, wavefunction, params, pot, field, Z=1)
imshow_time_frequency_characteristics(x, t, wavefunction, params, pot, field)

0
